In [1]:
######### 분석과제!
rm(list=ls())
### setwd

In [2]:
### packages
if(!require(caret)) install.packages("caret"); library(caret)
if(!require(e1071)) install.packages("e1071"); library(e1071)
if(!require(dplyr)) install.packages("dplyr"); library(dplyr)
if(!require(class)) install.packages("class"); library(class) # for knn
if(!require(kknn)) install.packages("kknn"); library(kknn) # for wknn
if(!require(data.table)) install.packages("data.table"); library(data.table)

Loading required package: caret
Loading required package: lattice
Loading required package: ggplot2
Loading required package: e1071
Loading required package: dplyr

Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: class
Loading required package: kknn

Attaching package: 'kknn'

The following object is masked from 'package:caret':

    contr.dummy

Loading required package: data.table

Attaching package: 'data.table'

The following objects are masked from 'package:dplyr':

    between, first, last



In [35]:
#### load data
pro.train <- fread('data/profiles_train.csv')
pro.train <- as.data.frame(pro.train)
pro.train$gen <- as.factor(pro.train$gen)
click.train <- fread('data/click_train.csv')
click.train <- as.data.frame(click.train)
pro.test <- fread('data/profiles_test.csv')
pro.test <- as.data.frame(pro.test)
click.test <- fread('data/click_test.csv')
click.test <- as.data.frame(click.test)
pro.train %>% head
click.train %>% head
str(click.train)

id,gen,job,resid
1003,여자,의료,서울
1006,남자,자영업,대전
1014,남자,자영업,경상
1025,여자,대학/대학원생,부산
1027,남자,전산직,경기
1039,남자,사무관리직,경기


id,time,st,st_c,st_t,cate
1003,2012072311,up.cafe.naver.com,1,0,커뮤니티
1003,2012080211,blog.naver.com,11,26,커뮤니티
1003,2012080712,cafe.naver.com,1,60,커뮤니티
1003,2012081011,blog.naver.com,59,346,커뮤니티
1003,2012082713,blog.daum.net,10,532,커뮤니티
1003,2012090711,blog.naver.com,39,93,커뮤니티


'data.frame':	1393986 obs. of  6 variables:
 $ id  : chr  "1003" "1003" "1003" "1003" ...
 $ time: int  2012072311 2012080211 2012080712 2012081011 2012082713 2012090711 2012091710 2012110615 2012110617 2012111012 ...
 $ st  : chr  "up.cafe.naver.com" "blog.naver.com" "cafe.naver.com" "blog.naver.com" ...
 $ st_c: int  1 11 1 59 10 39 8 1 1 7 ...
 $ st_t: int  0 26 60 346 532 93 12 53 60 805 ...
 $ cate: chr  "커뮤니티" "커뮤니티" "커뮤니티" "커뮤니티" ...


In [55]:
click.train$time = as.character(click.train$time)
click.train$time = strtrim(click.train$time,8)
click.train$time = as.numeric(click.train$time)
click.test$time = as.character(click.test$time)
click.test$time = strtrim(click.test$time,8)
click.test$time = as.numeric(click.test$time)

In [69]:
d <- click.train %>% group_by(id) %>% summarise(Day = length(unique(time)))
a <- click.train %>% group_by(id) %>% summarise(ST_T = sum(st_t)) # 총 체류시간
b <- click.train %>% group_by(id) %>% summarise(ST_C = sum(st_c)) # 총 페이지본 횟수
c <- click.train %>% group_by(id) %>% summarise(CATE = length(unique(cate)))


'data.frame':	1393986 obs. of  6 variables:
 $ id  : chr  "1003" "1003" "1003" "1003" ...
 $ time: num  20120723 20120802 20120807 20120810 20120827 ...
 $ st  : chr  "up.cafe.naver.com" "blog.naver.com" "cafe.naver.com" "blog.naver.com" ...
 $ st_c: int  1 11 1 59 10 39 8 1 1 7 ...
 $ st_t: int  0 26 60 346 532 93 12 53 60 805 ...
 $ cate: chr  "커뮤니티" "커뮤니티" "커뮤니티" "커뮤니티" ...


In [70]:
pro.train$st_t = as.numeric(a$ST_T)
pro.train$st_c = as.numeric(b$ST_C)
pro.train$cov = as.numeric(c$CATE/22)
pro.train$day = as.numeric(d$Day)
pro.train = within(pro.train, rm("job","resid", "id"))

Warning message in rm("job", "resid", "id"):
"객체 'job'를 찾을 수 없습니다"Warning message in rm("job", "resid", "id"):
"객체 'resid'를 찾을 수 없습니다"Warning message in rm("job", "resid", "id"):
"객체 'id'를 찾을 수 없습니다"

In [71]:
pro.train %>% str()

'data.frame':	402 obs. of  5 variables:
 $ gen : Factor w/ 2 levels "남자","여자": 2 1 1 2 1 1 2 1 1 1 ...
 $ st_t: num  885021 138740 337567 426651 354698 ...
 $ st_c: num  34024 10651 9288 28349 12539 ...
 $ cov : num  0.773 0.818 0.864 0.864 0.818 ...
 $ day : num  129 129 185 107 194 223 130 166 150 126 ...


In [72]:
d <- click.test %>% group_by(id) %>% summarise(Day = length(unique(time)))
a <- click.test %>% group_by(id) %>% summarise(ST_T = sum(st_t)) # 총 체류시간
b <- click.test %>% group_by(id) %>% summarise(ST_C = sum(st_c)) # 총 페이지본 횟수
c <- click.test %>% group_by(id) %>% summarise(CATE = length(unique(cate)))
# pro.test$gen = as.numeric(pro.test$gen)-1
pro.test$st_t = as.numeric(a$ST_T)
pro.test$st_c = as.numeric(b$ST_C)
pro.test$cov = as.numeric(c$CATE/22)
pro.test$day = as.numeric(d$Day)
pro.test = within(pro.test, rm("job","resid", "id"))
pro.test %>% str()

Warning message in rm("job", "resid", "id"):
"객체 'job'를 찾을 수 없습니다"Warning message in rm("job", "resid", "id"):
"객체 'resid'를 찾을 수 없습니다"Warning message in rm("job", "resid", "id"):
"객체 'id'를 찾을 수 없습니다"

'data.frame':	99 obs. of  4 variables:
 $ st_t: num  230324 161998 137625 474261 730673 ...
 $ st_c: num  10102 8368 6750 16146 29484 ...
 $ cov : num  0.818 0.591 0.682 0.909 0.909 ...
 $ day : num  113 147 197 111 123 117 108 113 213 113 ...


In [73]:
## min-max 스케일링
normalize <- function(x){
  return( (x-min(x))/(max(x)-min(x)) )
}

In [74]:
pro.train.normal <- as.data.frame(lapply(pro.train[-1], normalize))
pro.test.normal <- as.data.frame(lapply(pro.test[-3], normalize))


In [75]:
pro.train.normal$gen = pro.train$gen
pro.train.normal$cov = pro.train$cov
pro.test.normal$cov = pro.test$cov

In [76]:
str(pro.train.normal)

'data.frame':	402 obs. of  5 variables:
 $ st_t: num  0.2044 0.0302 0.0766 0.0974 0.0806 ...
 $ st_c: num  0.412 0.122 0.105 0.342 0.145 ...
 $ cov : num  0.773 0.818 0.864 0.864 0.818 ...
 $ day : num  0.195 0.195 0.414 0.109 0.449 ...
 $ gen : Factor w/ 2 levels "남자","여자": 2 1 1 2 1 1 2 1 1 1 ...


In [77]:
pro.train.normal %>% head
pro.test.normal %>% head

st_t,st_c,cov,day,gen
0.20435473,0.4120312,0.7727273,0.1953125,여자
0.03024735,0.1219500,0.8181818,0.1953125,남자
0.07663370,0.1050339,0.8636364,0.4140625,남자
0.09741700,0.3415990,0.8636364,0.1093750,여자
0.08063036,0.1453819,0.8181818,0.4492188,남자
0.28894979,0.4882344,1.0000000,0.5625000,남자


st_t,st_c,day,cov
0.08878565,0.11385648,0.07608696,0.8181818
0.05415057,0.08897594,0.26086957,0.5909091
0.04179568,0.06575983,0.53260870,0.6818182
0.21243955,0.20057969,0.06521739,0.9090909
0.34241715,0.39196189,0.13043478,0.9090909
0.07811622,0.28124776,0.09782609,0.5909091


In [81]:
set.seed(1)
# createDataPartition
# caret패키지 안에 있는 데이터 파티션, 층화추출
idx <- createDataPartition(y = pro.train.normal$gen, p = 0.7, list =FALSE)
train1 <- pro.train.normal[idx,]
test1 <- pro.train.normal[-idx,]

In [84]:
train1

,st_t,st_c,cov,day,gen
3,0.07663370,0.10503388,0.8636364,0.41406250,남자
4,0.09741700,0.34159903,0.8636364,0.10937500,여자
6,0.28894979,0.48823442,1.0000000,0.56250000,남자
8,0.19578749,0.39882592,0.9545455,0.33984375,남자
9,0.02459052,0.03978951,0.6818182,0.27734375,남자
11,0.04665721,0.15020975,0.8636364,0.16406250,남자
13,0.07150716,0.14967607,0.8181818,0.09375000,여자
14,0.16943392,0.44108521,0.8181818,0.22265625,여자
15,0.19990547,0.51202621,0.8636364,0.15234375,여자
16,0.11019134,0.22039368,0.8181818,0.10156250,남자


In [88]:
repCv <- trainControl(method = "repeatedcv", number = 6,repeats = 4, verbose = T)

In [89]:
train.lda <- train(gen~., train1, method = "lda", trControl = repCv)

+ Fold1.Rep1: parameter=none 
- Fold1.Rep1: parameter=none 
+ Fold2.Rep1: parameter=none 
- Fold2.Rep1: parameter=none 
+ Fold3.Rep1: parameter=none 
- Fold3.Rep1: parameter=none 
+ Fold4.Rep1: parameter=none 
- Fold4.Rep1: parameter=none 
+ Fold5.Rep1: parameter=none 
- Fold5.Rep1: parameter=none 
+ Fold6.Rep1: parameter=none 
- Fold6.Rep1: parameter=none 
+ Fold1.Rep2: parameter=none 
- Fold1.Rep2: parameter=none 
+ Fold2.Rep2: parameter=none 
- Fold2.Rep2: parameter=none 
+ Fold3.Rep2: parameter=none 
- Fold3.Rep2: parameter=none 
+ Fold4.Rep2: parameter=none 
- Fold4.Rep2: parameter=none 
+ Fold5.Rep2: parameter=none 
- Fold5.Rep2: parameter=none 
+ Fold6.Rep2: parameter=none 
- Fold6.Rep2: parameter=none 
+ Fold1.Rep3: parameter=none 
- Fold1.Rep3: parameter=none 
+ Fold2.Rep3: parameter=none 
- Fold2.Rep3: parameter=none 
+ Fold3.Rep3: parameter=none 
- Fold3.Rep3: parameter=none 
+ Fold4.Rep3: parameter=none 
- Fold4.Rep3: parameter=none 
+ Fold5.Rep3: parameter=none 
- Fold5.Re

In [90]:
predict.lda <- predict(train.lda, test1)
confusionMatrix(predict.lda, test1$gen)

Confusion Matrix and Statistics

          Reference
Prediction 남자 여자
      남자   65   38
      여자   11    6
                                          
               Accuracy : 0.5917          
                 95% CI : (0.4982, 0.6805)
    No Information Rate : 0.6333          
    P-Value [Acc > NIR] : 0.8511442       
                                          
                  Kappa : -0.0096         
 Mcnemar's Test P-Value : 0.0002038       
                                          
            Sensitivity : 0.8553          
            Specificity : 0.1364          
         Pos Pred Value : 0.6311          
         Neg Pred Value : 0.3529          
             Prevalence : 0.6333          
         Detection Rate : 0.5417          
   Detection Prevalence : 0.8583          
      Balanced Accuracy : 0.4958          
                                          
       'Positive' Class : 남자            
                                          

In [ ]:
# a 앙상블할 모델의 갯수
###  b,c,d ... : method
### 1. 과제서 제시한 3개의 모델 우선!
### 2. 나아가서 일반화 및 파라매터 조정도 해봅시다!
voteClassifier <- function(a,b,c,d) {
  # 학습 부분, caret패키지 함수로 해주세요!
  
  # 예측 부분
  
  # predict count 부분
  
  # 최종 결과물 반환
  return()
}
predict.ensemble <- voteClassifier()
confusionMatrix(predict.ensemble, test.set)

#### 최종 결과물 제출
pred.test <- predict(~~모델, pro.test)
pred.test <- as.data.frame(pred.test)
names(pred.test) <- "gen"
write.csv(pred.test,'본인이름.csv',row.names =F)
###